In [ ]:
import pandas as pd
import numpy as np
import os

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram

from skopt import BayesSearchCV

from utils import ModelSuplier, DataLoader, DataSaver

In [2]:
ms = ModelSuplier()

pipelines = ms.pipelines

In [3]:
dl = DataLoader()

data_as_X_and_y = dl.transformed_data

Using default path


In [5]:
ds = DataSaver(os.path.join("..", "history_bayes"))

# Bayes Search (for default params)

In [ ]:
param_distributions = [
    {
        "model__max_depth": Integer(1, 31),
        "model__min_samples_split": Integer(2, 61),
        "model__criterion": Categorical(["gini", "entropy"]),
        "model__min_samples_leaf": Integer(1, 61)
    },
    {
        "model__n_estimators": Integer(100, 501),      
        "model__min_samples_leaf": Integer(1, 251),    
        "model__max_samples": Real(0.5, 0.5),        
        "model__max_features": Real(1e-6, 1 - 1e-6)   
    },
    {
        "model__max_depth": Integer(1, 20),
        "model__min_child_weight": Integer(0, 20),
        "model__eta": Real(0.01, 0.1), 
        "model__alpha": Real(1e-4, 10, prior="log-uniform")
    }   
]

NameError: name 'Integer' is not defined